<a href="https://colab.research.google.com/github/bjvvuuren/e_commerce_solutions/blob/main/Generate_Master_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'active-woodland-226211' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [2]:
import pandas as pd

In [3]:
query = """
WITH Demograhic_Order_data 
AS
  (
    SELECT
    A.user_id,
    A.gender,
    A.age,
    A.city,
    A.state, 
    A.country,
    A.traffic_source,
    B.order_id,
    B.purchase_date,
    B.qty
    
    FROM
    ((SELECT
      id as user_id,
      gender,
      age,
      city,
      state,
      country,
      traffic_source
      FROM `bigquery-public-data.thelook_ecommerce.users`) AS A

    FULL OUTER JOIN

    (
      SELECT
    order_id,
    user_id,
    created_at as purchase_date,
    num_of_item as qty,

    FROM `bigquery-public-data.thelook_ecommerce.orders`
    ) AS B

  on A.user_id = b.user_id
  )),
  Order_details AS
    (
      SELECT 
      D.order_id,
      D.user_id,
      D.product_id,
      D.status,
      D.revenue,
      C.product_name,
      C.product_brand,
      C.product_category
      FROM
      (
        SELECT
        id as product_id,
        name as product_name,
        brand as product_brand,
        department as product_category
        FROM `bigquery-public-data.thelook_ecommerce.products`
      ) AS C
      FULL OUTER JOIN
      (
        SELECT
        order_id,
        user_id,
        product_id,
        status,
        sale_price as revenue
        FROM `bigquery-public-data.thelook_ecommerce.order_items`) AS D
      
      ON C.product_id = d.product_id
    )
    


SELECT
E.purchase_date,
E.user_id,
E.order_id,
F.product_id,
E.gender as user_gender,
E.age as user_age,
E.city as user_city,
E.state as user_state, 
E.country as user_country,
E.traffic_source as user_traffic_source,
E.qty as order_qty,
F.status as order_status,
F.revenue,
F.product_name,
F.product_brand,
F.product_category

FROM
(
  (SELECT
  *
  FROM Demograhic_Order_data) as E

  FULL OUTER JOIN

  (
    SELECT
    *
    FROM Order_details ) as F

  ON
  E.user_id = F.user_id AND e.order_id = F.order_id
) 
ORDER BY E.purchase_date DESC
"""

user_purchases = pd.read_gbq(query, project)

In [4]:
user_purchases.to_pickle('user_purchases.pkl')